In [1]:
# Tiesinės regresijos veikimo principas - mūsų modelis stengsis nubrėžti tarp taškų liniją taip, kad iki jų būtų mažiausia įmanoma atstumų kvadratų suma.
# Vėliau, prognozuojant naujas reikšmes, jis remsis ta linija.
# Žinoma, esant daugiau negu dviems kintamiesiems, kaip šiuo atveju, x ir y, procesai pasidaro sudėtingi ir painūs, todėl turėkite omenyje, kad tai tik labai primityvus pavyzdys.

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

%matplotlib inline

In [3]:
mpg = sns.load_dataset('mpg')
mpg.head()

mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   
3  16.0          8         304.0       150.0    3433          12.0   
4  17.0          8         302.0       140.0    3449          10.5   

   model_year origin                       name  
0          70    usa  chevrolet chevelle malibu  
1          70    usa          buick skylark 320  
2          70    usa         plymouth satellite  
3          70    usa              amc rebel sst  
4          70    usa                ford torino

In [4]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model_year    398 non-null    int64  
 7   origin        398 non-null    object 
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


In [5]:
mpg[mpg['horsepower'].isnull()]

mpg  cylinders  displacement  horsepower  weight  acceleration  \
32   25.0          4          98.0         NaN    2046          19.0   
126  21.0          6         200.0         NaN    2875          17.0   
330  40.9          4          85.0         NaN    1835          17.3   
336  23.6          4         140.0         NaN    2905          14.3   
354  34.5          4         100.0         NaN    2320          15.8   
374  23.0          4         151.0         NaN    3035          20.5   

     model_year  origin                  name  
32           71     usa            ford pinto  
126          74     usa         ford maverick  
330          80  europe  renault lecar deluxe  
336          80     usa    ford mustang cobra  
354          81  europe           renault 18i  
374          82     usa        amc concord dl

In [6]:
mpg['horsepower'].loc[32] = 75
mpg['horsepower'].loc[126] = 105
mpg['horsepower'].loc[330] = 51
mpg['horsepower'].loc[336] = 120
mpg['horsepower'].loc[354] = 73
mpg['horsepower'].loc[374] = 82

C:\Users\zyman\AppData\Local\Temp\ipykernel_11088\3019380189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mpg['horsepower'].loc[32] = 75
C:\Users\zyman\AppData\Local\Temp\ipykernel_11088\3019380189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mpg['horsepower'].loc[126] = 105
C:\Users\zyman\AppData\Local\Temp\ipykernel_11088\3019380189.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mpg['horsepower'].loc[330] = 51
C:\U

In [7]:
mpg['origin'].unique()

array(['usa', 'japan', 'europe'], dtype=object)

In [8]:
dummies = pd.get_dummies(mpg['origin'])
dummies

europe  japan  usa
0         0      0    1
1         0      0    1
2         0      0    1
3         0      0    1
4         0      0    1
..      ...    ...  ...
393       0      0    1
394       1      0    0
395       0      0    1
396       0      0    1
397       0      0    1

[398 rows x 3 columns]

In [9]:
data = pd.concat([mpg, dummies], axis=1)
data.tail()

mpg  cylinders  displacement  horsepower  weight  acceleration  \
393  27.0          4         140.0        86.0    2790          15.6   
394  44.0          4          97.0        52.0    2130          24.6   
395  32.0          4         135.0        84.0    2295          11.6   
396  28.0          4         120.0        79.0    2625          18.6   
397  31.0          4         119.0        82.0    2720          19.4   

     model_year  origin             name  europe  japan  usa  
393          82     usa  ford mustang gl       0      0    1  
394          82  europe        vw pickup       1      0    0  
395          82     usa    dodge rampage       0      0    1  
396          82     usa      ford ranger       0      0    1  
397          82     usa       chevy s-10       0      0    1

In [10]:
data.drop(['origin', 'name'], axis=1, inplace=True)
data.tail()

mpg  cylinders  displacement  horsepower  weight  acceleration  \
393  27.0          4         140.0        86.0    2790          15.6   
394  44.0          4          97.0        52.0    2130          24.6   
395  32.0          4         135.0        84.0    2295          11.6   
396  28.0          4         120.0        79.0    2625          18.6   
397  31.0          4         119.0        82.0    2720          19.4   

     model_year  europe  japan  usa  
393          82       0      0    1  
394          82       1      0    0  
395          82       0      0    1  
396          82       0      0    1  
397          82       0      0    1

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model_year    398 non-null    int64  
 7   europe        398 non-null    uint8  
 8   japan         398 non-null    uint8  
 9   usa           398 non-null    uint8  
dtypes: float64(4), int64(3), uint8(3)
memory usage: 23.1 KB


In [17]:
features_names = data.columns[1:]
features_names
features = data[features_names]
features.tail()

cylinders  displacement  horsepower  weight  acceleration  model_year  \
393          4         140.0        86.0    2790          15.6          82   
394          4          97.0        52.0    2130          24.6          82   
395          4         135.0        84.0    2295          11.6          82   
396          4         120.0        79.0    2625          18.6          82   
397          4         119.0        82.0    2720          19.4          82   

     europe  japan  usa  
393       0      0    1  
394       1      0    0  
395       0      0    1  
396       0      0    1  
397       0      0    1

In [18]:
target = data['mpg']
target.tail(10)

388    26.0
389    22.0
390    32.0
391    36.0
392    27.0
393    27.0
394    44.0
395    32.0
396    28.0
397    31.0
Name: mpg, dtype: float64

In [15]:
# pip install scikit_learn
from sklearn.model_selection import train_test_split

In [19]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.33, random_state=42)

In [23]:
features_train.head(), target_train.head()

(     cylinders  displacement  horsepower  weight  acceleration  model_year  \
 324          4          85.0        65.0    2110          19.2          80   
 176          6         232.0        90.0    3211          17.0          75   
 119          4         114.0        91.0    2582          14.0          73   
 192          6         250.0       105.0    3353          14.5          76   
 202          6         258.0        95.0    3193          17.8          76   
 
      europe  japan  usa  
 324       0      1    0  
 176       0      0    1  
 119       1      0    0  
 192       0      0    1  
 202       0      0    1  ,
 324    40.8
 176    19.0
 119    20.0
 192    22.0
 202    17.5
 Name: mpg, dtype: float64)

In [24]:
from sklearn.linear_model import LinearRegression

In [29]:
mpg_reg_model = LinearRegression().fit(features_train, target_train)
target_prediction = mpg_reg_model.predict(features_test)
target_prediction

array([32.5807764 , 29.11531912, 20.82468337, 16.81042998, 12.53671407,
       26.89224642, 27.46096913,  9.71898846, 17.0760993 , 21.5306433 ,
       12.55239646, 35.97375981, 24.45309969, 13.64179325, 23.33981127,
        6.42582008, 29.07341977, 25.16152699, 16.28333751, 36.13277466,
       24.85161985, 18.54208742, 28.42495262, 28.10143028, 14.80888168,
       35.98685651, 26.59945344, 24.37634664, 21.67995077,  8.83362446,
       27.96549245, 36.05109578, 20.56982898, 25.76405355, 34.06919842,
       11.88189052, 22.51551402, 21.69716135, 12.45688148, 27.88145573,
       25.93878775, 28.4408266 , 21.35409858,  7.28439335, 24.25967981,
       34.75785888, 27.84102343, 25.38400728, 23.61343313, 28.72129644,
       21.97440896, 33.44984508, 32.52325432, 10.88379546, 27.44117061,
        9.91298837, 19.76560359, 29.76634903, 22.4060054 , 20.17165045,
       16.02114488, 31.77642227, 23.4149797 , 21.7584608 , 20.55449373,
       25.16026886, 24.6024376 , 31.80307564, 27.49403129, 12.71

In [30]:
target_prediction = pd.Series(data=target_prediction, name='mpg_guess')
target_prediction.head()

0    32.580776
1    29.115319
2    20.824683
3    16.810430
4    12.536714
Name: mpg_guess, dtype: float64

In [31]:
# duomenu palyginimas
target_comparison = pd.concat([target_test.reset_index(), target_prediction], axis=1)
target_comparison

index   mpg  mpg_guess
0      198  33.0  32.580776
1      396  28.0  29.115319
2       33  19.0  20.824683
3      208  13.0  16.810430
4       93  14.0  12.536714
..     ...   ...        ...
127    392  27.0  27.142602
128    379  36.0  32.114696
129    358  31.6  31.324449
130    310  38.1  34.694051
131    145  32.0  29.518624

[132 rows x 3 columns]

In [33]:
target_comparison = target_comparison.set_index('index')
target_comparison

mpg  mpg_guess
index                 
198    33.0  32.580776
396    28.0  29.115319
33     19.0  20.824683
208    13.0  16.810430
93     14.0  12.536714
...     ...        ...
392    27.0  27.142602
379    36.0  32.114696
358    31.6  31.324449
310    38.1  34.694051
145    32.0  29.518624

[132 rows x 2 columns]

In [39]:
final_comparison = pd.concat([target_comparison, mpg['name']], axis=1)
final_comparison[final_comparison['mpg'] > 0]

mpg  mpg_guess                        name
198  33.0  32.580776                 honda civic
396  28.0  29.115319                 ford ranger
33   19.0  20.824683                 amc gremlin
208  13.0  16.810430  plymouth volare premier v8
93   14.0  12.536714    plymouth fury gran sedan
..    ...        ...                         ...
392  27.0  27.142602            chevrolet camaro
379  36.0  32.114696              mercury lynx l
358  31.6  31.324449                   mazda 626
310  38.1  34.694051       toyota corolla tercel
145  32.0  29.518624                  datsun 710

[132 rows x 3 columns]

In [40]:
mpg_reg_model.score(features_test, target_test)

0.8463742656811793